In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from peakweather.dataset import PeakWeatherDataset

In [ ]:
# Download the data in the current working directory
ds = PeakWeatherDataset(
    root=None,
    compute_uv=False,
    station_type="meteo_station",
    freq="h",
    aggregation_methods={'temperature': 'mean'},
)

In [ ]:
# Choose a station and parameter for time series analysis
station = "ABO"  # Example station ID
parameter = "temperature"

# Get training and test data
train = ds.get_observations(stations=station, parameters=parameter, first_date="2020-01-01", last_date="2020-12-31")
test = ds.get_observations(stations=station, parameters=parameter, first_date="2021-01-01", last_date="2021-01-31")

# Visualize the training data
train.plot(title=f"Training Data: {parameter} at station {station}", ylabel=f"{parameter}", xlabel="Date", figsize=(12,6), legend=False)
plt.show()

In [ ]:
# Remove periodic component (daily seasonality)
diff_train = train.diff().dropna()

# Visualize the de-seasonalized training data
diff_train.plot(title=f"De-seasonalized training Data: {parameter} at station {station}", ylabel=f"{parameter}", xlabel="Date", figsize=(12,6), legend=False)
plt.show()

# Look at ACF and PACF plots of de-seasonalized data
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plot_acf(diff_train, ax=plt.gca(), lags=24)
plt.title("ACF (auto-correlation)")

plt.subplot(1,2,2)
plot_pacf(diff_train, ax=plt.gca(), lags=24, method='ywm')
plt.title("PACF (partial auto-correlation)")
plt.tight_layout()
plt.show()

"""
Hints:
- If PACF cuts off after lag k (significant at lag 1..k then small after), consider p=k.
- If ACF cuts off after lag k, consider q=k.
"""

In [ ]:
# Choose values for p, d, q based on ACF/PACF analysis
p,d,q,P,D,Q,s = 2, 0, 0, 1, 0, 0, 24  # Example values
# Fit SARIMA model
print(f"Fitting SARIMA({p},{d},{q})x({P},{D},{Q})_{s} ...")
model = SARIMAX(endog=train, order=(p,d,q), seasonal_order=(P,D,Q,s)).fit()
print(model.summary())

In [ ]:
w, h = max((p,d,q,s+P,s+D,s+Q)), 24
maes = []
windows = []
horizons = []
forecasts = []
for i in range(w, len(test) - h):
    window = test[i-w:i]
    horizon = test[i:i+h]
    # Create a new SARIMA model with the same structure
    forecast = SARIMAX(endog=window, order=model.model.order, seasonal_order=model.model.seasonal_order).filter(model.params).get_forecast(steps=h).predicted_mean
    mae = np.abs(horizon[(station, parameter)].values - forecast.values).mean()
    maes.append(mae)
    windows.append(window)
    horizons.append(horizon)
    forecasts.append(forecast)
print(f"Mean MAE over test set: {np.mean(maes):.4f}")

In [ ]:
sample = 200

plt.figure(figsize=(12,4))
plt.plot(windows[sample].index, windows[sample], label='input window')
plt.plot(horizons[sample].index, horizons[sample], label='true horizon')
plt.plot(forecasts[sample].index, forecasts[sample], label='forecast')
plt.legend()
plt.title(f"SARIMA({p},{d},{q})x({P},{D},{Q})_{s} Forecast vs True Horizon")
plt.show()